# Códigos Utilizados

En este documento se presetan los gráficos de la integración de los distintos parámetros del modelo cosmológico Cos-Cosh para determinar y clasificar la singularidad que se presenta a futuro, según los análisis analíticos descritos en el Trabajo de Titulación

- V. Silva

### Librerías

Importamos las librerías que usaremos para la realización de los gráficos e inegraciones.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

### Definición de parámetros cosmológicos

Definimos los parámetros de acuerdo al Cuadro 3.1 expuesto en el trabajo de titulación con $a_0 = 1$ correspondiente al presente cosmológico.

Dado que vamos a estudiar evoluciones temporales de los parámetros nos coviene tenere las unidades en tiempo cosmológico (Gyr) por lo que podemos hacer una conversión (un pequeño truco) dimensional para $H_0$.

Esta conversión es puramente dimensional y se realiza mediante la equivalencia

$$
\begin{align}
1\,\text{Mpc} &= 3.085677581 \times 10^{19}\,\text{km}\\
1\,\text{Gyr} &= 3.15576 \times 10^{16}\,\text{s}\\
1.022712 \times 10^{-3}\,\text{Gyr}^{-1} &\simeq 1\,\text{km s}^{-1}\,\text{Mpc}^{-1} \\
1\,\text{Gyr}^{-1} &= 977.792221\,\text{km s}^{-1}\,\text{Mpc}^{-1}
\end{align}
$$

De modo que

$$
H_0^{(\text{Gyr}^{-1})}
= \frac{H_0^{(\text{km s}^{-1}\,\text{Mpc}^{-1})}}{977.792221}.
$$


Con esto garantizamos la consistencia dimensional en expresiones como $H_0 (t - t_0)$ que aparecen en la solución formal del factor de escala.


In [2]:
wa = 5.203
wb = 0.188
wc = 3.634
a0 = 1.0

H0_km_s_Mpc = 69.91
Gyr_Inv = 977.792221
H0_Gyr_inv = H0_km_s_Mpc / Gyr_Inv

### Definición del modelo

Ahora definimos la ecuacion de estado de la energía oscura $\omega(a)$ tal como se ve en la parametrización introducida en el Capítulo 3 del trabajo:

$$
\omega(a) = \omega_a \left(\frac{a}{a_0}\right)\cos\!\left(\frac{a}{a_0}\right)
      - \omega_b \left(\frac{a}{a_0}\right)\cosh\!\left(\frac{a}{a_0}\right)
      - \omega_c,
$$

Para resolver la evolución cosmológica, se introduce la variable independiente

$$
x = \ln a,
$$

y las variables dependientes

$$
G(x), \qquad \tau(x) = H_0\, t,
$$

donde $G(a)$ es la función auxiliar (de la cual se puede saber más si así lo desea consultando los apuntes del TdT) definida por

$$
G(a) = \int (1 + \omega(a))\, d\ln a.
$$

En términos de estas variables, el sistema de ecuaciones diferenciales toma la forma
$$
\frac{dG}{dx} = 1 + \omega(a),
$$
$$
\frac{d\tau}{dx} = \exp\!\left(\frac{3}{2} G\right),
$$
donde se ha utilizado la relación
$$
E(a) \equiv \frac{H(a)}{H_0} = \exp\!\left(-\frac{3}{2} G(a)\right).
$$

Esto lo implementamos de manera numérica con la función `rhs`, que devuelve el lado derecho de las ecuaciones diferenciales.


### En referencia al uso del modelo completo

Si tenemos y se ha mostrado en el trabajo que hay una expersión asintótica para la ecuación de estado en el régimen $a \gg a_0$,
$$
w(a) \approx -\frac{w_b}{2}\frac{a}{a_0} e^{a/a_0} - w_c,
$$
esta aproximación solo es válida en el límite estrictamente futuro (y muy futuro, véase como dijimos $a \gg a_0$ )y no describe de manera fiable la dinámica cosmológica en el entorno del presente ($a \sim a_0$).

Dado que la función auxiliar (que se puede consultar en la ecuación 3.33 del TdT o en el anexo del mismo para mayor profundidad)
$$
G(a) = \int (1 + w(a))\, d\ln a
$$
acumula la historia completa de la ecuación de estado, el uso de la forma reducida introduciría errores sistemáticos en la integración temporal del factor de escala $a(t)$.

Por este motivo, la evolución cosmológica se resuelve utilizando la expresión completa de $w(a)$, mientras que la forma asintótica se emplea únicamente para el análisis del comportamiento futuro y la clasificación de singularidades.


In [3]:
def w_de(a: float) -> float:
    x = a / a0
    return wa * x * np.cos(x) - wb * x * np.cosh(x) - wc

# dG/dx = 1+w(a)
# dtau/dx = exp(1.5 G)   (porque tau=H0*t y E=H/H0=exp(-1.5G))
EXP_CLIP = 700.0 # 700 porque está justo por debajo del límite de overflow en float64
def rhs(x, y):
    G, tau = y
    a = np.exp(x)
    dGdx = 1.0 + w_de(a)
    expo = np.clip(1.5*G, -EXP_CLIP, EXP_CLIP)
    dtaudx = np.exp(expo)
    return [dGdx, dtaudx]

Para la evaluación numérica de
$$
\frac{d\tau}{dx} = \exp\!\left(\frac{3}{2} G\right),
$$
introducimos un término para tener "seguridad numérica" `EXP_CLIP`, que limita el argumento de la función exponencial a un rango finito. Esta operación evita desbordamientos numéricos asociados al crecimiento rápido de $G$ en escenarios phantom, sin modificar la dinámica física del modelo en nuestro régimen de interés.

### Criterios de detención de la integración

Durante la integración del sistema dinámico se introducen condiciones de detención que permiten identificar de manera eficiente el comportamiento asintótico del tiempo cosmológico, evitando integraciones innecesarias o numéricamente inestables.

Primero, definimos un umbral negativo $G_{\text{stop}}$ tal que, cuando la función auxiliar $G(a)$ alcanza valores suficientemente negativos, se cumple que

$$
\frac{d\tau}{dx} = \exp\!\left(\frac{3}{2}G\right) \simeq 0
$$

En este régimen, el incremento del tiempo adimensional $\tau = H_0 t$ se vuelve despreciable y la integral temporal ya converge, indicando la existencia de un tiempo futuro finito $t$. La integración se detiene automáticamente cuando se llega a $G = G_{\text{stop}}$.

Además, integramos un segundo criterio basado en un valor máximo permitido para $\tau$. Si durante la integración el tiempo adimensional crece sin cota y supera un umbral muy grande $\tau_{\max}$, esto constituye evidencia numérica robusta de que el tiempo futuro diverge, es decir, $t \to \infty$. En este caso, la integración se detiene para evitar problemas de estabilidad numérica.


Teniendo estos dos criterios podemos distinguir de forma eficiente y optimizada.

In [4]:
# si G cae muy muy negativo, entonces dtau/dx=exp(1.5G) ~ 0
# => el integral de tiempo ya convergió (t es finito).
G_STOP = -60.0
def ev_G_stop(x, y):  # 0 cuando G=G_STOP
    return y[0] - G_STOP
ev_G_stop.terminal = True
ev_G_stop.direction = -1

# si tau se gigante, es nuestra evidencia fuerte de que t => infinito
TAU_MAX = 1e6
def ev_tau_max(x, y):
    return TAU_MAX - y[1]
ev_tau_max.terminal = True
ev_tau_max.direction = -1

### Integración por etapas y diagnóstico del “destino” cosmológico

Para estudiar el comportamiento futuro del modelo se integra el sistema en la variable $x=\ln a$, con condición inicial en el presente cosmológico:

$$
x_0=\ln a_0,\qquad y_0=[G(x_0),\tau(x_0)]=[0,0],
$$

donde $G(a_0)=0$ corresponde a la normalización $\rho_{\rm de}(a_0)=\rho_0$ y $\tau(a_0)=H_0 t_0=0$ fija el origen temporal.

La integración se realiza hacia el futuro aumentando progresivamente el límite superior $a_{\max}$ en etapas:

$$
a_{\max}\in \{5,10,20,50,100,300,10^3,10^4,10^5\}.
$$

Este esquema por tramos evita inestabilidades numéricas y permite detener la simulación tan pronto como se identifique el régimen asintótico relevante.

Una vez obtenidas $G$ y $\tau$, se reconstruyen las magnitudes físicas principales:

$$
a = e^x,\qquad t\,[\text{Gyr}] = \frac{\tau}{H_0\,[\text{Gyr}^{-1}]}
$$

$$
\frac{\rho}{\rho_0}=\exp(-3G),\qquad \frac{H}{H_0}=\exp\!\left(-\frac{3}{2}G\right)
$$

y además

$$
\omega(a)=\omega_{\rm de}(a),\qquad \frac{p}{\rho_0}=\omega\,\frac{\rho}{\rho_0}
$$

Al final damos cuenta de la razón de detención (con `stop_reason`) junto con los valores finales alcanzados por $a$, $\tau$ y $t$, lo que permite clasificar el comportamiento asintótico del modelo.

In [5]:
x0 = np.log(a0)
y0 = [0.0, 0.0]

# aumentamos a_max por etapas, sin reventar, hasta que converge (G_STOP), o diverge tiempo (TAU_MAX), o llega a un límite de a.
a_stages = [5, 10, 20, 50, 100, 300, 1e3, 1e4, 1e5]
method = "Radau"

x_all, G_all, tau_all = [], [], []

x_start, y_start = x0, y0
stop_reason = None

for a_max in a_stages:
    x_max = np.log(a_max)
    sol = solve_ivp(
        rhs, (x_start, x_max), y_start,
        method=method, rtol=1e-10, atol=1e-12,
        events=[ev_G_stop, ev_tau_max],
        dense_output=True
    )
    if not sol.success:
        raise RuntimeError("solve_ivp falló: " + sol.message)

    # guardamos el tramo
    x_seg = sol.t
    G_seg = sol.y[0]
    tau_seg = sol.y[1]

    # para que al concatenar no se nos duplique el primer punto
    if len(x_all) > 0:
        x_seg = x_seg[1:]
        G_seg = G_seg[1:]
        tau_seg = tau_seg[1:]

    x_all.append(x_seg)
    G_all.append(G_seg)
    tau_all.append(tau_seg)

    # vemos cuál de nuestras condiciones se activa
    if len(sol.t_events[0]) > 0:
        stop_reason = f"Convergencia de tiempo (G cruzó {G_STOP})"
        break
    if len(sol.t_events[1]) > 0:
        stop_reason = f"Tiempo diverge (tau alcanzó {TAU_MAX})"
        break

    # siguiente etepa
    x_start = sol.t[-1]
    y_start = [sol.y[0, -1], sol.y[1, -1]]

if stop_reason is None:
    stop_reason = f"No ocurrió ningún evento (último a_max={a_stages[-1]})"

x = np.concatenate(x_all)
G = np.concatenate(G_all)
tau = np.concatenate(tau_all)

a = np.exp(x)
t_Gyr = tau / H0_Gyr_inv

rho = np.exp(-3.0 * G)      # rho/rho0
Hn  = np.exp(-1.5 * G)      # H/H0
w   = np.array([w_de(ai) for ai in a])
p   = w * rho               # p/rho0

print("\nResultados:")
print("  stop_reason:", stop_reason)
print(f"  a_final   = {a[-1]:.3e}")
print(f"  tau_final = {tau[-1]:.6e}")
print(f"  t_final   = {t_Gyr[-1]:.6f} Gyr")


Resultados:
  stop_reason: Convergencia de tiempo (G cruzó -60.0)
  a_final   = 6.298e+00
  tau_final = 4.138075e-01
  t_final   = 5.787695 Gyr


In [6]:
t_finito = ("Convergencia de tiempo" in stop_reason)
t_inf = t_Gyr[-1]  # si converge, esto es ≈ t_infinto

if t_finito:
    print(f"\n=> t FINITO = {t_inf:.6f} Gyr") # esto sería nuestro t_s, t_s siendo el tiempo en el que ocurre la singularidad
else:
    print("\n=> t NO finito en este diagnóstico (t -> infinito)")


=> t FINITO = 5.787695 Gyr


### Construcción de curvas para la clasificación

Para identificar y clasificar el tipo de comportamiento asintótico del modelo (y eventualmente la presencia de singularidades), se construyen curvas en función del tiempo cosmológico.

Se consideran las funciones:

*   $a(t)$
*   $\rho(t)$
*   $|p(t)|$
*   $H(t)$





In [7]:
# graficaremos a(t), rho(t), |p(t)|, H(t)
# si t es finito, graficamos hasta 0.995 t
# si es infinito, graficamos hasta un t_plot razonable (como por ejemplo 50 Gyr o t_final si es menor)
if t_finito:
    t_plot_max = 0.995 * t_inf
else:
    t_plot_max = min(50.0, t_Gyr[-1])

# Invertir a(t)
a_of_t = interp1d(t_Gyr, a, kind="linear", bounds_error=False, fill_value="extrapolate")
G_of_a = interp1d(a, G, kind="cubic", bounds_error=False, fill_value="extrapolate")

t_plot = np.linspace(0.0, t_plot_max, 1600)
a_plot = a_of_t(t_plot)
G_plot = G_of_a(a_plot)

rho_plot = np.exp(-3.0 * G_plot)
H_plot = np.exp(-1.5 * G_plot)
w_plot = np.array([w_de(ai) for ai in a_plot])
p_plot = w_plot * rho_plot


print("="*80)
print(f"  t_final_plot = {t_plot[-1]:.6f} Gyr")
print(f"  a_end        = {a_plot[-1]:.3e}")
print(f"  rho_end      = {rho_plot[-1]:.3e}")
print(f"  |p|_end      = {np.abs(p_plot[-1]):.3e}")
print(f"  H_end/H0     = {H_plot[-1]:.3e}")
print(f"  w_end        = {w_plot[-1]:.3e}")
print("="*80)

  t_final_plot = 5.758756 Gyr
  a_end        = 2.072e+00
  rho_end      = 8.446e+02
  |p|_end      = 8.767e+03
  H_end/H0     = 2.906e+01
  w_end        = -1.038e+01


### Los Gráficos

Hacemos los plot de cada uno de los parámetros que buscamos de forma separada y los guardamos en un directorio para su posterior uso.

In [8]:
# Hacemos un directorio para guardar las imágenes de los plots
import os
if not os.path.exists('plots'):
    os.makedirs('plots')

# a(t)
plt.figure(figsize=(14, 10))
plt.plot(t_plot, a_plot, linewidth=3.5, color='royalblue')
plt.xlabel("t [Gyr]", fontsize=16, fontweight="bold")
plt.ylabel("a(t)", fontsize=16, fontweight="bold")
plt.title("Factor de escala a(t)\n" +
          f"H0={H0_km_s_Mpc:.2f} km/s/Mpc",
          fontsize=18, fontweight="bold")
plt.grid(True, alpha=0.3)
if t_finito:
    plt.axvline(t_inf, linestyle="--", linewidth=2.5, color='red', alpha=0.7)
    plt.text(t_inf-0.45, a_plot[-1]/2, f't_s = {t_inf:.2f} Gyr',
             fontsize=14, color='red', ha='center', va='bottom')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig("plots/a_of_t_CosCosh_H06991.png", dpi=300, bbox_inches="tight")
plt.close()
print("Guardado: plots/a_of_t_CosCosh_H06991.png")

# rho(t)
plt.figure(figsize=(14, 10))
plt.semilogy(t_plot, rho_plot, linewidth=3.5, color='darkgreen')
plt.xlabel("t [Gyr]", fontsize=16, fontweight="bold")
plt.ylabel("rho_DE(t)/rho_0", fontsize=16, fontweight="bold")
plt.title("Densidad de energia oscura rho(t)\n" +
          f"H0={H0_km_s_Mpc:.2f} km/s/Mpc",
          fontsize=18, fontweight="bold")
plt.grid(True, which="both", alpha=0.3)
if t_finito:
    plt.axvline(t_inf, linestyle="--", linewidth=2.5, color='red', alpha=0.7)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig("plots/rho_of_t_CosCosh_H06991.png", dpi=300, bbox_inches="tight")
plt.close()
print("Guardado: plots/rho_of_t_CosCosh_H06991.png")

# |p(t)|
plt.figure(figsize=(14, 10))
plt.semilogy(t_plot, np.abs(p_plot), linewidth=3.5, color='darkorange')
plt.xlabel("t [Gyr]", fontsize=16, fontweight="bold")
plt.ylabel("|p_DE(t)|/rho_0", fontsize=16, fontweight="bold")
plt.title("Presion de energia oscura (modulo) |p(t)|\n" +
          f"H0={H0_km_s_Mpc:.2f} km/s/Mpc",
          fontsize=18, fontweight="bold")
plt.grid(True, which="both", alpha=0.3)
if t_finito:
    plt.axvline(t_inf, linestyle="--", linewidth=2.5, color='red', alpha=0.7)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig("plots/p_of_t_CosCosh_H06991.png", dpi=300, bbox_inches="tight")
plt.close()
print("Guardado: plots/p_of_t_CosCosh_H06991.png")

# H(t)
plt.figure(figsize=(14, 10))
plt.semilogy(t_plot, H_plot, linewidth=3.5, color='purple')
plt.xlabel("t [Gyr]", fontsize=16, fontweight="bold")
plt.ylabel("H(t)/H_0", fontsize=16, fontweight="bold")
plt.title("Parametro de Hubble normalizado H(t)/H_0\n" +
          f"H0={H0_km_s_Mpc:.2f} km/s/Mpc",
          fontsize=18, fontweight="bold")
plt.grid(True, which="both", alpha=0.3)
if t_finito:
    plt.axvline(t_inf, linestyle="--", linewidth=2.5, color='red', alpha=0.7)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig("plots/H_of_t_CosCosh_H06991.png", dpi=300, bbox_inches="tight")
plt.close()
print("Guardado: plots/H_of_t_CosCosh_H06991.png")

# w(t)
plt.figure(figsize=(14, 10))
plt.plot(t_plot, w_plot, linewidth=3.5, color='crimson')
plt.xlabel("t [Gyr]", fontsize=16, fontweight="bold")
plt.ylabel("w(t)", fontsize=16, fontweight="bold")
plt.title("Ecuacion de estado efectiva w(t)\n" +
          f"H0={H0_km_s_Mpc:.2f} km/s/Mpc",
          fontsize=18, fontweight="bold")
plt.grid(True, alpha=0.3)
if t_finito:
    plt.axvline(t_inf, linestyle="--", linewidth=2.5, color='red', alpha=0.7)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig("plots/w_of_t_CosCosh_H06991.png", dpi=300, bbox_inches="tight")
plt.close()
print("Guardado: plots/w_of_t_CosCosh_H06991.png")

print("="*80)
print("Todo guardado en 'plots/'")
print("="*80)

Guardado: plots/a_of_t_CosCosh_H06991.png
Guardado: plots/rho_of_t_CosCosh_H06991.png
Guardado: plots/p_of_t_CosCosh_H06991.png
Guardado: plots/H_of_t_CosCosh_H06991.png
Guardado: plots/w_of_t_CosCosh_H06991.png
Todo guardado en 'plots/'
